In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer 
import nltk.corpus
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, merge, LSTM, Lambda, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
!pip install ipython-autotime
%load_ext google.colab.data_table
%load_ext autotime
nltk.download('stopwords')
stop = stopwords.words('english')
ps = PorterStemmer()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=179cb63fa46c04735a982f7c1bf58f4301c3f272b77881e5c521e0ccaabbfd24
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from nltk.corpus import stopwords

import re
# Descomentar el git para cargar el archivo train.csv que se encuentra en el repositorio
!git clone https://github.com/iampukar/toxic-comments-classification.git


Cloning into 'toxic-comments-classification'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.
time: 16.8 s


In [3]:
train = pd.read_csv('/content/toxic-comments-classification/train.csv')

time: 685 ms


In [4]:
train.head(10) 

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


time: 48.4 ms


In [5]:
num_data = 5000
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

time: 3.89 ms


#prepocesamiento  de los datos

In [6]:
# solo seleccion los primeros num_data elementos de la base de datos
train = train[: num_data]

time: 1.21 ms


In [7]:
def clean(comment):

    comment = comment.lower()
    comment = re.sub(r"\n", " ", comment)
    re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", comment)
    re.sub(r"\d+", "", comment)
    output = ""
    first = True

    for element in comment.split():

        if element not in stop and len(element) > 1:
            
            output += element + " " if first else " " + element

            first = False
    
    return output


def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].apply(clean)
    
    return df


data_clean = clean_text(train, 'comment_text', 'text_clean')



#data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


time: 811 ms


In [8]:
data_clean.head(10) 

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,text_clean
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore meta...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww! matches background colour i'm seemingl...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"hey man, i'm really trying edit war. guy cons..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,can't make real suggestions improvement wonde...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"you, sir, hero. chance remember page that's on?"
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"congratulations well, use tools well. talk"
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker piss around work
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,vandalism matt shirvington article reverted. ...
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"sorry word 'nonsense' offensive you. anyway, ..."
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,alignment subject contrary dulithgow


time: 27.3 ms


In [9]:
vectorizer = CountVectorizer ()

time: 720 µs


In [10]:
""" En esta renglon se realiza la vectorizacion de las muestras tomando cada palabra en la bolsa de palabras
    como una caracteristicas. Tal que cada muestra se representa como un vector de n caracteristicas tomando
    valores de 0 o 1 si incluyesen tales caracteristicas (o palabras) en la muestra.
"""
X = vectorizer.fit_transform(data_clean['text_clean'])
y = data_clean[['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']][: num_data]


time: 196 ms


Aplicando PCA a los datos

In [11]:
pca = PCA (0.95)


time: 950 µs


In [12]:
XX = pca.fit_transform( X.toarray())

time: 3min 31s


In [13]:
print(X.shape)
nb_words = (np.max(XX) + 1)


(5000, 24026)
time: 6.08 ms


#Entrenamiento sin PCA

In [14]:
x_l, x_test, y_l, y_test = train_test_split(X, y, test_size = 0.2, train_size = 0.8)
X_train, x_cv, y_train, y_cv = train_test_split(x_l, y_l, test_size = 0.1, train_size = 0.9)

time: 11.9 ms


In [15]:
model_noPca = Sequential()
model_noPca.add(Dense(500, input_dim=X.shape[1], activation='sigmoid' ))
model_noPca.add(Dense(400,  activation='relu' ))
model_noPca.add(Dense(100,  activation='relu' ))
model_noPca.add(Dense(6, activation='softmax'))
#model_noPca.add(Dense(500, input_dim=XX.shape[1], activation='sigmoid' ))


model_noPca.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

time: 8.38 s


In [16]:
X_train.shape

(3600, 24026)

time: 3.38 ms


In [17]:
model_noPca.fit(X_train, y_train, validation_data=(x_cv, y_cv), epochs=10, batch_size=32)

Train on 3600 samples, validate on 400 samples
Epoch 1/10
3600/3600 [==============================] - 3s 820us/step - loss: 0.2393 - accuracy: 0.9559 - val_loss: 0.2636 - val_accuracy: 0.9454
Epoch 2/10
3600/3600 [==============================] - 1s 344us/step - loss: 0.2304 - accuracy: 0.9636 - val_loss: 0.2553 - val_accuracy: 0.9454
Epoch 3/10
3600/3600 [==============================] - 1s 344us/step - loss: 0.2279 - accuracy: 0.9636 - val_loss: 0.2546 - val_accuracy: 0.9454
Epoch 4/10
3600/3600 [==============================] - 1s 345us/step - loss: 0.2250 - accuracy: 0.9652 - val_loss: 0.2563 - val_accuracy: 0.9454
Epoch 5/10
3600/3600 [==============================] - 1s 340us/step - loss: 0.2222 - accuracy: 0.9642 - val_loss: 0.2562 - val_accuracy: 0.9462
Epoch 6/10
3600/3600 [==============================] - 1s 339us/step - loss: 0.2194 - accuracy: 0.9665 - val_loss: 0.2554 - val_accuracy: 0.9450
Epoch 7/10
3600/3600 [==============================] - 1s 340us/step - loss:

time: 14.5 s


In [18]:
y_pred = model_noPca.predict(x_test)
y_ = y_test

time: 121 ms


In [19]:
y_.values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0]])

time: 3.34 ms


Generando la matriz de confusion con cada etiqueta

In [20]:
for i in range(y.shape[1]):
    
    cm = confusion_matrix(y_.values[:, i], y_pred[:, i] > 0.5)
    total = sum(cm[0]) + sum(cm[1])
    answered = cm[0][0] + cm[1][1]

    print(f"{train.columns[i+2]}\nAcurracy: {answered/total}\n{cm}\n")

toxic
Acurracy: 0.883
[[864  24]
 [ 93  19]]

severe_toxic
Acurracy: 0.992
[[992   0]
 [  8   0]]

obscene
Acurracy: 0.946
[[943   0]
 [ 54   3]]

threat
Acurracy: 0.997
[[997   0]
 [  3   0]]

insult
Acurracy: 0.94
[[940   0]
 [ 60   0]]

identity_hate
Acurracy: 0.991
[[991   0]
 [  9   0]]

time: 25.7 ms


#Entrenamiento usando PCA

In [21]:
x_l, x_test, y_l, y_test = train_test_split(XX, y, test_size = 0.2, train_size = 0.8)
X_train, x_cv, y_train, y_cv = train_test_split(x_l, y_l, test_size = 0.1, train_size = 0.9)

time: 47.1 ms


In [29]:
X_train.shape

(3600, 838)

time: 5.41 ms


In [22]:
model = Sequential()
model.add(Dense(500, input_dim=XX.shape[1], activation='sigmoid' ))
model.add(Dense(400,  activation='relu' ))
model.add(Dense(100,  activation='relu' ))
model.add(Dense(6, activation='softmax'))
#model.add(Dense(500, input_dim=XX.shape[1], activation='sigmoid' ))


model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

time: 50.5 ms


In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 500)               419500    
_________________________________________________________________
dense_6 (Dense)              (None, 400)               200400    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               40100     
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 606       
Total params: 660,606
Trainable params: 660,606
Non-trainable params: 0
_________________________________________________________________
time: 1.88 ms


In [24]:
model.fit(X_train, y_train, validation_data=(x_cv, y_cv), epochs=10, batch_size=32)

Train on 3600 samples, validate on 400 samples
Epoch 1/10
3600/3600 [==============================] - 0s 121us/step - loss: 0.2461 - accuracy: 0.9520 - val_loss: 0.2251 - val_accuracy: 0.9683
Epoch 2/10
3600/3600 [==============================] - 0s 96us/step - loss: 0.2331 - accuracy: 0.9618 - val_loss: 0.2212 - val_accuracy: 0.9683
Epoch 3/10
3600/3600 [==============================] - 0s 97us/step - loss: 0.2302 - accuracy: 0.9604 - val_loss: 0.2179 - val_accuracy: 0.9683
Epoch 4/10
3600/3600 [==============================] - 0s 100us/step - loss: 0.2283 - accuracy: 0.9626 - val_loss: 0.2195 - val_accuracy: 0.9683
Epoch 5/10
3600/3600 [==============================] - 0s 93us/step - loss: 0.2267 - accuracy: 0.9639 - val_loss: 0.2192 - val_accuracy: 0.9671
Epoch 6/10
3600/3600 [==============================] - 0s 94us/step - loss: 0.2245 - accuracy: 0.9637 - val_loss: 0.2175 - val_accuracy: 0.9688
Epoch 7/10
3600/3600 [==============================] - 0s 94us/step - loss: 0.22

time: 4.1 s


In [25]:
model.evaluate(x_test, y_test)

1000/1000 [==============================] - 0s 44us/step


[0.2460953993797302, 0.9546666145324707]

time: 49.1 ms


In [26]:
y_pred = model.predict(XX)
y_ = y.values

time: 149 ms


In [27]:
print(y_pred.shape)
print(y_.shape)

(5000, 6)
(5000, 6)
time: 1.63 ms


Generando la matriz de confusion con cada etiqueta

In [28]:
for i in range(y.shape[1]):
    cm = confusion_matrix(y_[:, i], y_pred[:, i] > 0.5)
    total = sum(cm[0]) + sum(cm[1])
    answered = cm[0][0] + cm[1][1]

    print(f"{train.columns[i+2]}\nAcurracy: {answered/total}\n{cm}\n")

toxic
Acurracy: 0.9096
[[4473   24]
 [ 428   75]]

severe_toxic
Acurracy: 0.9898
[[4949    0]
 [  51    0]]

obscene
Acurracy: 0.948
[[4729    1]
 [ 259   11]]

threat
Acurracy: 0.996
[[4980    0]
 [  20    0]]

insult
Acurracy: 0.9466
[[4729    0]
 [ 267    4]]

identity_hate
Acurracy: 0.9902
[[4951    0]
 [  49    0]]

time: 80.6 ms


## Resultados del entrenamiento utilizando PCA
>results | Toxic |  Severe toxic | Obscene | Threat | Insult | Identity hate
>--- | --- | --- | ---| --- | --- |  --- | 
>Acurracy | 0.908 | 0.988 | 0.94766 | 0.995 | 0.948 | 0.989
>Precision | 0.908| 0.987| 0.947| 0.995| 0.954| 0.989
>True positive |2689 | 2962| 2832 | 2986 | 2815 | 2968
>False negative| 4 | 0 | 1 | 0 | 21 | 0
>True negative| 35 | 2 | 11 | 0 | 29 | 0
>False positive| 272 | 36 | 156 | 14 | 135 | 32
> GPU | 2.76 s
> None | 7.2seg

## Resultados del entrenamiento sin utilizar PCA
>results | Toxic |  Severe toxic | Obscene | Threat | Insult | Identity hate
>--- | --- | --- | ---| --- | --- |  --- | 
>Acurracy | 0.921 | 0.987 | 0.943 | 0.995 | 0.947 | 0.989
>Precision | 0.975 | 0.999 | 1 | 1 | 0.998 | 1
>True positive |2962 | 2962| 2833 | 2986 | 2836 | 2968
>False negative| 1 | 0 | 0 | 0 | 0 | 0
>True negative| 234 | 37 | 167 | 14 | 159 | 32
>False positive| 73 | 1 | 0 | 0 | 5 | 0
> GPU | 6.78 s
> None | 1 min 2seg


# Conclusion

A pesar de que los resultados pareciera ser superiores en el modelo entrenado sin usar el reductor de caracteristicas PCA. El tiempo de entrenamiento es mayor en este modelo. Ademas de que el tiempo para su vectorizacion fue mucho mayor